# ResNet

## 一、为什么需要ResNet？

### 更深的网络不一定更好

```
正常的想法：
    10层网络 → 还行
    20层网络 → 更好
    50层网络 → 应该更更好？

实际情况：
    10层 → 还行
    20层 → 更好
    50层 → 反而变差了！

不是过拟合！是网络太深，梯度传不到底层，底层根本学不动
```

### ResNet怎么解决？

**加一条捷径，让输入可以跳过某些层直接传过去**

```
普通网络：
    x → 卷积 → 卷积 → 输出F(x)

ResNet：
    x → 卷积 → 卷积 → 结果F(x)
    x ─────────────────→ +       ← 把x加回来！
                          ↓
                    输出 = F(x) + x
```

### 为什么 +x 有用？

```
中间的层学到了东西：
    输出 = 有用的东西 + x → 比x更好 ✓

中间的层什么都没学到：
    输出 = 0 + x = x → 至少不会更差 ✓

所以加层只会变好或不变，永远不会变差
这就是为什么ResNet可以做到100层甚至1000层
```
---

## 二、残差连接长什么样？

```
输入x
  │
  ├─────────────────────────┐
  │                         │ 这条线就是"捷径"
  ↓                         │
Conv → BN → ReLU → Conv → BN   │
  │                         │
  ↓                         │
  + ←───────────────────────┘  把x加回来
  ↓
 ReLU
  ↓
 输出
```

**整个ResNet就是把很多这样的块堆在一起**

---

## 三、ResNet的整体结构

```
和LeNet套路一样：

LeNet：  Conv → Pool → Conv → Pool → Flatten → FC
ResNet： Conv → 残差块×很多 → 全局Pool → Flatten → FC

区别只是中间换成了残差块
```

### 数据怎么变的

```
通道越来越多：  64 → 128 → 256 → 512
空间越来越小：  56 → 28  → 14  → 7
最后：全局池化 → Flatten → Linear

跟LeNet一模一样的思路！
```

---

## 四、不同版本

```
ResNet18：18层，最快
ResNet34：34层，常用
ResNet50：50层，更强但更慢

数字 = 层数，越大越强越慢
实际中最常用：ResNet18 或 ResNet34
```

---

## 五、代码实现

### 直接用PyTorch自带的（实际中用）

```python
import torchvision.models as models

net = models.resnet18()    # 拿到ResNet18的结构（随机权重）
```

---

## 六、怎么训练？和LeNet完全一样！

### 唯一要改的：最后一层

```python
# ResNet18默认输出1000类（ImageNet的类别数）
# 你的任务可能是10类，需要改一下

net = models.resnet18()
net.fc = nn.Linear(512, 10)    # 改成你的类别数
```

**为什么要改？**

```
PyTorch自带的ResNet是按ImageNet设计的（1000类）
你的数据集可能只有10类
最后一层的输出大小必须等于你的类别数
所以要手动改一下
```

### 完整训练代码

```python
import torch
from torch import nn
import torchvision.models as models

# ========== 第1步：定义网络 ==========
net = models.resnet18()   # 拿到ResNet结构
net.fc = nn.Linear(512, 10) # 改最后一层为10类

# ========== 第2步：搬到GPU ==========
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net.to(device)

# ========== 第3步：定义优化器和损失 ==========
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)
loss_fn = nn.CrossEntropyLoss()

# ========== 第4步：训练（和LeNet一模一样！） ==========
for epoch in range(num_epochs):
    net.train()
    for X, y in train_loader:
        X, y = X.to(device), y.to(device)
        y_hat = net(X)
        loss = loss_fn(y_hat, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # 评估
    net.eval()
    # ...
```

### 对比LeNet的训练代码

```python
# LeNet
net = nn.Sequential(
    nn.Conv2d(1, 6, kernel_size=5, padding=2),
    nn.Sigmoid(), nn.AvgPool2d(2, stride=2),
    nn.Conv2d(6, 16, kernel_size=5),
    nn.Sigmoid(), nn.AvgPool2d(2, stride=2),
    nn.Flatten(),
    nn.Linear(16*5*5, 120), nn.Sigmoid(),
    nn.Linear(120, 84), nn.Sigmoid(),
    nn.Linear(84, 10),
)

# ResNet
net = models.resnet18()
net.fc = nn.Linear(512, 10)

# 后面的训练代码：完全一样！
# optimizer, loss_fn, for循环, backward, step...全一样
```

**区别只在定义网络那两行，训练代码一个字都不用改！**

---

## 七、什么时候改最后一层的512？

```python
net.fc = nn.Linear(512, 10)
#                  ^^^
#                  这个512哪来的？
```

```
ResNet18/34：最后一层输入是512   → nn.Linear(512, 类别数)
ResNet50/101/152：最后一层输入是2048 → nn.Linear(2048, 类别数)

不确定的话：
    print(net.fc)
    # 会告诉你 Linear(in_features=???, out_features=1000)
    # 那个???就是你要用的数字
```

---

## 八、总结

### 概念

```
ResNet = 每隔几层加一条捷径（输出 = F(x) + x）
好处：网络再深也能训练
是现代深度学习的标配
```

### 代码模板

```python
# 定义网络
net = models.resnet18()          # 或resnet34, resnet50
net.fc = nn.Linear(512, 类别数)   # 改最后一层

# 训练：和之前完全一样
net.to(device)
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)
loss_fn = nn.CrossEntropyLoss()
# for循环训练...
```

### 要记住的

```
1. ResNet的核心 = Y += X（残差连接）
2. 实际中直接用 models.resnet18()，不用自己写
3. 记得改最后一层 net.fc = nn.Linear(512, 你的类别数)
4. 训练代码和LeNet完全一样，一个字不用改
```